In [1]:
import pandas as pd
import re
import string
from imblearn.over_sampling import SMOTE

Using TensorFlow backend.
C:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as

In [2]:
data = pd.read_excel('filter_out_lat_sh.xlsx',sheetname='Sheet1')

In [3]:
data.columns

Index(['Call type', 'Callfixdate', 'Callnumber', 'Comments', 'Logdate',
       'Problem Summary', 'Query type', 'Resolver group', 'extracted_problem',
       'pattern_used', 'Call type:Query type', 'predicted_value'],
      dtype='object')

In [4]:
data = data[['extracted_problem','Call type:Query type']]

data.columns = ['Extracted Problem Summary','Call type:Query type']

data_new_2nd = data.copy()

In [5]:
data_new_2nd.head()

Extracted Problem Summary  \
0           summary : cmrs stuck in sys-admin error.   
1  notification number   notification subject    ...   
2  Subject: Your Call has been resolved - 02IT792...   
3                 summary: nhs mail password reset     
4                     summary: nhs mail password       

            Call type:Query type  
0      Oracle:CMR Workflow Error  
1      Oracle:CMR Workflow Error  
2  AURORA:Password Reset Request  
3        NHS Mail:Password Reset  
4        NHS Mail:Password Reset

In [6]:
def collect_text(row):
    data1 = row['Extracted Problem Summary']
    #print data1
    #tokens = nltk.wordpunct_tokenize(data)
    #print (type(data1))
    if type(data1) not in [int,float]:
        #print (type(data1))
        #line = re.sub(r"[^A-Za-z\s]", " ", data1.strip())
#         line = []
#         for i in data1.split():
#             result = re.findall(r"[a-zA-z]+",i)
#             if len(result)!=0 and len(result[0])==len(i):
#                 line.append(i)
#         line = ' '.join(line)
#         line = re.sub('issue summary'," ",line)
#         line = re.sub('issue'," ",line)
#         line = re.sub('subject'," ",line)
        #line = re.sub(" \d+", " ", data1.strip())
        line = data1.strip()
        line = line.translate(str.maketrans('', '', string.punctuation))
        line = line.strip()
        tokens = line.split()
    else:
        tokens=[]
    return ' '.join(tokens)

In [7]:
temp_new=data_new_2nd[['Extracted Problem Summary']].apply(collect_text,axis=1)
data_new_2nd['Extracted Problem Summary'] = temp_new
data_new_2nd.head()

Extracted Problem Summary  \
0               summary cmrs stuck in sysadmin error   
1  notification number notification subject 43619...   
2  Subject Your Call has been resolved 02IT792356...   
3                    summary nhs mail password reset   
4                          summary nhs mail password   

            Call type:Query type  
0      Oracle:CMR Workflow Error  
1      Oracle:CMR Workflow Error  
2  AURORA:Password Reset Request  
3        NHS Mail:Password Reset  
4        NHS Mail:Password Reset

In [8]:
print (len(data))
data_new_2nd.dropna(inplace=True)
len(data_new_2nd)

9543


9543

## Create Model

In [9]:
data_new_2nd.groupby(by=['Call type:Query type']).count().sort_values('Extracted Problem Summary',ascending=False)

Extracted Problem Summary
Call type:Query type                                                        
NHS Mail:Shared Mailbox                                                  958
Outlook:Fault/Query                                                      823
Oracle:Workflow Request                                                  746
Oracle:Process Query                                                     735
Business Intelligence:BI Report Query                                    716
Oracle:User Amend                                                        670
NHS Mail:Password Reset                                                  509
Oracle:Workflow Rule                                                     494
Oracle:User End Date                                                     460
Oracle:General Ledger Query                                              433
AURORA:Password Reset Request                                            381
Oracle:Reports Query                                                     359
Oracle:iProc/eProc Requisitions Query                                    356
Oracle:CMR Workflow Error                                                345
Oracle:Accounts Payable Query                                            174
Oracle:User New Setup                                                    152
Oracle:JRE/JInitiator Query                                              138
vQSM:Operational Query                                                   132
Skype for Business:Fault/Query                                           111
Outlook:Archive                                                           99
PC:Performance Issue                                                      98
Oracle:CMR Cancellation                                                   95
Oracle:CMR Workflow Request                                               84
Oracle:Non PO Rules Request                                               65
Business Intelligence:BI Login Issue - Individual                         64
CRM System:User End Date                                                  51
Kofax MailManager:Fault/Query                                             45
vQSM:Password Reset Request                                               40
Oracle:User Request Bulk Change                                           40
IT Intranet:Password Reset                                                30
Network:External Website Access Issues                                    29
CRM System:Password Reset Request                                         25
Kofax KTM:Performance Issue                                               24
Beehive:Password Reset Request                                            18
RPA-Kapow:Fault/Query                                                     17
RPA-UiPath:Fault/Query                                                    16
Azure:Fault/Query                                                         11

In [10]:

# list_classes = ["Oracle:CMR Workflow Error","Oracle:User New Setup","NHS Mail:Shared Mailbox","Outlook:Fault/Query",
#                 "Oracle:R12 Login Assistance","Oracle:Workflow Request","Business Intelligence:BI Report Query",
#                 "Oracle:Process Query","Oracle:Password Reset Request","NHS Mail:Password Reset","Oracle:General Ledger Query",
#                 "Oracle:Workflow Rule","Oracle:Reports Query","Oracle:iProc/eProc Requisitions Query","Oracle:User Amend",
#                 "Oracle:Log In Problem Individual","NHS Mail:Fault/Query","Oracle:User End Date","AURORA:Password Reset Request",
#                 "Oracle:Accounts Payable Query","Oracle:JRE/JInitiator Query","vQSM:Operational Query","Outlook:Archive","PC:Performance Issue"
#                 ]


# list_classes.extend(['Oracle:CMR Workflow Request','Oracle:CMR Cancellation','Business Intelligence:BI Login Issue - Individual',
#                      'Oracle:Non PO Rules Request','CRM System:User End Date','Kofax MailManager:Fault/Query',
#                      'Oracle:User Request Bulk Change','vQSM:Password Reset Request','Network:External Website Access Issues',
#                      'CRM System:Password Reset Request','IT Intranet:Password Reset','Kofax KTM:Performance Issue',
#                      'RPA-UiPath:Fault/Query','Beehive:Password Reset Request','RPA-Kapow:Fault/Query','Azure:Fault/Query',
#                      'Manual Payments:Fault/Query','Web Form:Performance Issue'])



In [11]:
#data_new_2nd.dropna(inplace=True)

In [12]:
# lis=[]
# for i in data_new_2nd.values:
#     if i[1] not in list_classes:
#         continue
    
#     lis.append(i)

In [13]:
#len(data_new_2nd),len(lis)

In [14]:
# data_new_2nd=pd.DataFrame(lis,columns=data_new_2nd.columns)
# data_new_2nd.drop_duplicates(inplace=True)
# len(data_new_2nd)

In [15]:
#len(pd.unique(data_new_2nd['Call type:Query type'])),len(list_classes)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier,ExtraTreesClassifier,AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn import model_selection
from sklearn.metrics import confusion_matrix,f1_score,accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBRFClassifier
import pickle

In [41]:
tfidf = TfidfVectorizer(ngram_range=(1,3))
#model = SelectFromModel(RandomForestClassifier(n_estimators=100),n_jobs=-1)

In [44]:
#X = tfidf.fit_transform(data_new_2nd['Extracted Problem Summary'].values)
#y = data_new_2nd['Call type:Query type'].values

X = tfidf.fit_transform(data_new_2nd['Extracted Problem Summary'].values)
y = data_new_2nd['Call type:Query type'].values

X_old = X.copy()
y_old = y.copy()

print (X.shape,y.shape)
sm = SMOTE(random_state=2,k_neighbors=3)
X,y = sm.fit_sample(X, y)

#X = model.fit_transform(X,y)

print (X.shape,y.shape)

(9543, 353915) (9543,)
(35446, 353915) (35446,)


In [45]:
# skf = StratifiedKFold(n_splits=2)
# skf.get_n_splits(X, y)

# for train_index, test_index in skf.split(X, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]
    
#     clf = linear_model.LogisticRegression(class_weight='balanced',solver='lbfgs',C=10,max_iter=300)
#     clf.fit(X_train,y_train)
    
#     predict = clf.predict(X_test)
#     print ('F1-Score - ',f1_score(y_test,predict,average='weighted'))
#     print ('Accuracy - ',accuracy_score(y_test,predict))
#     #print (confusion_matrix(y_test,predict))

clf = linear_model.LogisticRegression(class_weight='balanced',solver='lbfgs',C=10,max_iter=300)
#clf = SVC(kernel='linear',class_weight='balanced',gamma=0.1)
#clf = XGBRFClassifier(max_depth=5,n_jobs=-1)
#clf = ExtraTreesClassifier(n_estimators=100,n_jobs=-1)
clf.fit(X,y)

C:\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=10, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=300, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [46]:
predict = clf.predict(X_old)
print ('F1-Score - ',f1_score(y_old,predict,average='weighted'))
print ('Accuracy - ',accuracy_score(y_old,predict))
ls=[]
for i,j in zip(y,predict):
    if i == j:
        ls.append([i,j,True])
    else:
        ls.append([i,j,False])
    

pd.DataFrame(ls,columns=['Actual','Predict','Equals']).to_excel('./my_cleaned_data/train_result.xlsx',index=0)

F1-Score -  0.9670954874245896
Accuracy -  0.966257990149848


In [47]:
f=open('lr_model.pickle','wb')
pickle.dump(clf,f)
f.close()

# f=open('smote_model.pickle','wb')
# pickle.dump(sm,f)
# f.close()

# f=open('feature_model.pickle','wb')
# pickle.dump(model,f)
# f.close()

f=open('pipeline.pickle','wb')
pickle.dump(tfidf,f)
f.close()

In [48]:
del tfidf,clf,predict

## Testing

In [49]:
f=open('lr_model.pickle','rb')
clf=pickle.load(f)
f.close()

# f=open('smote_model.pickle','rb')
# sm=pickle.load(f)
# f.close()

# f=open('feature_model.pickle','rb')
# model=pickle.load(f)
# f.close()

f=open('pipeline.pickle','rb')
pipeline=pickle.load(f)
f.close()

In [50]:
data = pd.read_excel('NHS_MAY1_with_33queryType -changed.xlsx',index_col=0)
data = data[['extracted_problem','Call type:Query type']]

data.columns = ['Extracted Problem Summary','Call type:Query type']

f=open('lr_model.pickle','rb')
clf=pickle.load(f)
f.close()

# f=open('smote_model.pickle','rb')
# sm=pickle.load(f)
# f.close()

# f=open('feature_model.pickle','rb')
# model=pickle.load(f)
# f.close()

f=open('pipeline.pickle','rb')
pipeline=pickle.load(f)
f.close()

In [51]:
list_include = ['Oracle:CMR Workflow Error', 'AURORA:Password Reset Request',
        'NHS Mail:Password Reset', 'NHS Mail:Shared Mailbox',
        'Oracle:User End Date', 'Oracle:User Amend',
        'Kofax KTM:Performance Issue', 'Oracle:User New Setup',
        'Business Intelligence:BI Report Query', 'Oracle:Process Query',
        'Oracle:Workflow Request', 'Oracle:Reports Query',
        'Outlook:Fault/Query', 'Skype for Business:Fault/Query',
        'Kofax MailManager:Fault/Query', 'Oracle:CMR Cancellation',
        'Oracle:Workflow Rule', 'Oracle:JRE/JInitiator Query',
        'Oracle:General Ledger Query', 'vQSM:Operational Query',
        'Outlook:Archive', 'PC:Performance Issue',
        'Oracle:CMR Workflow Request', 'Oracle:Accounts Payable Query',
        'IT Intranet:Password Reset',
        'Oracle:iProc/eProc Requisitions Query',
        'Oracle:Non PO Rules Request', 'Oracle:User Request Bulk Change',
        'vQSM:Password Reset Request',
        'Business Intelligence:BI Login Issue - Individual',
        'Beehive:Password Reset Request', 'RPA-Kapow:Fault/Query',
        'Network:External Website Access Issues', 'Azure:Fault/Query',
        'RPA-UiPath:Fault/Query', 'CRM System:User End Date',
        'CRM System:Password Reset Request']



lis = []
for i in data.values:
    if i[1] in list_include:
        lis.append(i)

data = pd.DataFrame(lis,columns=data.columns)

In [52]:
temp=data[['Extracted Problem Summary']].apply(collect_text,axis=1)
data['Extracted Problem Summary'] = temp

In [53]:
data.dropna(inplace=True)
X_test = pipeline.transform(data['Extracted Problem Summary'].values)
y_test = data['Call type:Query type'].values
print (X_test.shape)


(2206, 353915)


In [54]:
predict = clf.predict(X_test)
print ('F1-Score - ',f1_score(y_test,predict,average='weighted'))
print ('Accuracy - ',accuracy_score(y_test,predict))
#print (confusion_matrix(y,predict))

ls=[]
for i,j in zip(y,predict):
    if i == j:
        ls.append([i,j,True])
    else:
        ls.append([i,j,False])
    

pd.DataFrame(ls,columns=['Actual','Predict','Equals']).to_excel('./my_cleaned_data/test_result.xlsx',index=0)

F1-Score -  0.8008647315157648
Accuracy -  0.7923844061650045


C:\anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [55]:
lis=[]
for i in pd.unique(data['Call type:Query type'].values):
    test = data[data['Call type:Query type']==i]
    print ('***************** for ',i,' ****************************',' sample length ',len(test))
    predict = clf.predict(pipeline.transform(test['Extracted Problem Summary'].values))
    print ('F1-Score - ',f1_score(test['Call type:Query type'].values,predict,average='weighted'))
    print ('Accuracy - ',accuracy_score(test['Call type:Query type'].values,predict))
    
    if accuracy_score(test['Call type:Query type'].values,predict)<0.5:
        lis.append(i)
    #print (confusion_matrix(test['Call type:Query type'].values,predict))

***************** for  Oracle:CMR Workflow Error  ****************************  sample length  396
F1-Score -  0.9268292682926829
Accuracy -  0.8636363636363636
***************** for  Oracle:User Request Bulk Change  ****************************  sample length  14
F1-Score -  0.4444444444444445
Accuracy -  0.2857142857142857
***************** for  Kofax MailManager:Fault/Query  ****************************  sample length  5


C:\anaconda\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


F1-Score -  0.5714285714285715
Accuracy -  0.4
***************** for  Oracle:Process Query  ****************************  sample length  67
F1-Score -  0.8547008547008548
Accuracy -  0.746268656716418
***************** for  NHS Mail:Shared Mailbox  ****************************  sample length  106
F1-Score -  0.8842105263157896
Accuracy -  0.7924528301886793
***************** for  Oracle:User Amend  ****************************  sample length  405
F1-Score -  0.9453124999999999
Accuracy -  0.8962962962962963
***************** for  Business Intelligence:BI Login Issue - Individual  ****************************  sample length  18
F1-Score -  0.7142857142857143
Accuracy -  0.5555555555555556
***************** for  Oracle:User New Setup  ****************************  sample length  159
F1-Score -  0.8045112781954886
Accuracy -  0.6729559748427673
***************** for  Oracle:User End Date  ****************************  sample length  180
F1-Score -  0.8679245283018869
Accuracy -  0.7666666

C:\anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


F1-Score -  0.4444444444444445
Accuracy -  0.2857142857142857
***************** for  Kofax KTM:Performance Issue  ****************************  sample length  5
F1-Score -  0.5714285714285715
Accuracy -  0.4
***************** for  RPA-UiPath:Fault/Query  ****************************  sample length  4
F1-Score -  0.0
Accuracy -  0.0
***************** for  Outlook:Archive  ****************************  sample length  10
F1-Score -  0.7499999999999999
Accuracy -  0.6
***************** for  Beehive:Password Reset Request  ****************************  sample length  3
F1-Score -  0.8000000000000002
Accuracy -  0.6666666666666666
***************** for  Oracle:Non PO Rules Request  ****************************  sample length  5
F1-Score -  0.0
Accuracy -  0.0
***************** for  Oracle:CMR Cancellation  ****************************  sample length  9
F1-Score -  0.9411764705882353
Accuracy -  0.8888888888888888
***************** for  PC:Performance Issue  ****************************  samp

In [56]:
pd.DataFrame(lis)

0
0          Oracle:User Request Bulk Change
1            Kofax MailManager:Fault/Query
2                     Oracle:Reports Query
3                 CRM System:User End Date
4              Oracle:CMR Workflow Request
5   Network:External Website Access Issues
6        CRM System:Password Reset Request
7              Kofax KTM:Performance Issue
8                   RPA-UiPath:Fault/Query
9              Oracle:Non PO Rules Request
10                       Azure:Fault/Query
11                   RPA-Kapow:Fault/Query